# XCH4 dataset satellite lev2

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr
import xesmf as xe
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")

## Define request

In [ ]:
# Time parameters
year_start = 2005
year_stop = 2006

collection_id = "satellite-methane"
request = {
    "processing_level": "level_2",
    "variable": "xch4",
    "sensor_and_algorithm": "merged_emma",
    "version": "4.4",
    "year": [str(year) for year in range(year_start, year_stop + 1)],
    "month": [f"{i:02d}" for i in range(1, 13)],
    "day": [f"{i:02d}" for i in range(1, 32)],
}

## Define function to cache

In [ ]:
def monthly_regrid(ds, d_lon, d_lat, method, **kwargs):
    """Resample to monthly regular grid.

    Parameters
    ----------
    ds: Dataset
        Dataset to resample
    d_lon, d_lat: int
        Longitude/latitude step size
    method: str
        xesmf method
    **kwargs: Any
        Keyword arguments for xesmf

    Returns
    -------
    Dataset
    """
    kwargs.setdefault("locstream_in", True)
    ds = ds.chunk(time=-1).set_coords(["longitude", "latitude"])
    grid_out = xe.util.grid_global(d_lon=d_lon, d_lat=d_lat, cf=True)
    ds_out = ds.resample(time="1MS").map(
        diagnostics.regrid, grid_out=grid_out, method=method, **kwargs
    )
    ds_out["time"].attrs["standard_name"] = "time"
    return ds_out.where(ds_out["contributing_algorithms"] > 0)

## Download and transform data

In [ ]:
ds = download.download_and_transform(
    collection_id,
    request,
    chunks={"year": 1},
    transform_func=monthly_regrid,
    transform_func_kwargs={"d_lon": 1, "d_lat": 1, "method": "nearest_d2s"},
)

## Compute seasonal anomalies

In [ ]:
climatology = diagnostics.seasonal_weighted_mean(ds)
with xr.set_options(keep_attrs=True):
    anomaly = (
        ds.groupby("time.year").map(diagnostics.seasonal_weighted_mean) - climatology
    )
for da in anomaly.data_vars.values():
    if "long_name" in da.attrs:
        da.attrs["long_name"] += " Anomaly"

## Plot seasonal anomalies

In [ ]:
plot.projected_map(
    anomaly["xch4"], projection=ccrs.Robinson(), col="season", row="year"
)